##https://data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15057440

In [90]:
# Python 샘플 코드 #
from urllib.request import Request, urlopen
# Request https://docs.python.org/ko/3/library/urllib.request.html#urllib.request.Request
# urlopen https://docs.python.org/ko/3/library/urllib.request.html#urllib.request.urlopen

In [91]:
from urllib.parse import urlencode, quote_plus
# urlenocde https://docs.python.org/ko/3/library/urllib.parse.html#urllib.parse.urlencode
# quote_plus https://docs.python.org/ko/3/library/urllib.parse.html#urllib.parse.quote_plus

In [92]:
from urllib.parse import unquote

In [93]:
from xml.etree.ElementTree import parse

In [94]:
import requests

In [127]:
#KEY = 'yEaR%2F3MDedRSlVJL%2F2pxnVg0yre1N5VF3RZ%2FUAt56MJ7J2mNpfqhUvy05pXV0uhHTVY7DbyCR8xmMaDdYga67Q%3D%3D'
KEY = 'M%2B4%2FqUiadT8X8PhgFjaQLDu%2BIOgPMURfGsOX%2FmVxwHQVJgnVR%2FMPjDYXkuQNwUFbZXlfnX5Lls3SUCiCLIFjgQ%3D%3D'
API_KEY = requests.utils.unquote(KEY)
API_KEY = unquote(KEY)
print(API_KEY)

M+4/qUiadT8X8PhgFjaQLDu+IOgPMURfGsOX/mVxwHQVJgnVR/MPjDYXkuQNwUFbZXlfnX5Lls3SUCiCLIFjgQ==


In [128]:
# getBusPosByRtid
# 노선ID로 차량들의 위치정보를 조회한다
# 서울
url = 'http://ws.bus.go.kr/api/rest/buspos/getBusPosByRtid'
queryParams = '?' + urlencode({ quote_plus('ServiceKey') : API_KEY, quote_plus('busRouteId') : '204000046' })

In [142]:
# 경유정류소목록조회 buslocationservice
# 해당 노선이 정차하는 경유정류소 목록과 정류소명, 중앙차로여부, 회차점, 좌표값 등을 제공한다.
url_k = 'http://openapi.gbis.go.kr/ws/rest/buslocationservice'
queryParams_k = '?' + urlencode({ quote_plus('serviceKey') : API_KEY, quote_plus('routeId') : '204000046' })

In [143]:
# 버스도착정보조회 서비스
url1 = 'http://ws.bus.go.kr/api/rest/arrive/getArrInfoByRouteAll'
queryParams1 = '?' + urlencode({ quote_plus('ServiceKey') : API_KEY, quote_plus('busRouteId') : '204000046' })

In [144]:
print(url + queryParams)

http://ws.bus.go.kr/api/rest/buspos/getBusPosByRtid?ServiceKey=M%2B4%2FqUiadT8X8PhgFjaQLDu%2BIOgPMURfGsOX%2FmVxwHQVJgnVR%2FMPjDYXkuQNwUFbZXlfnX5Lls3SUCiCLIFjgQ%3D%3D&busRouteId=204000046


In [145]:
print(url1 + queryParams1)

http://ws.bus.go.kr/api/rest/arrive/getArrInfoByRouteAll?ServiceKey=M%2B4%2FqUiadT8X8PhgFjaQLDu%2BIOgPMURfGsOX%2FmVxwHQVJgnVR%2FMPjDYXkuQNwUFbZXlfnX5Lls3SUCiCLIFjgQ%3D%3D&busRouteId=204000046


In [146]:
print(url_k + queryParams_k)

http://openapi.gbis.go.kr/ws/rest/buslocationservice?serviceKey=M%2B4%2FqUiadT8X8PhgFjaQLDu%2BIOgPMURfGsOX%2FmVxwHQVJgnVR%2FMPjDYXkuQNwUFbZXlfnX5Lls3SUCiCLIFjgQ%3D%3D&routeId=204000046


In [153]:
buslocation = url + queryParams1
busarrive = url1 + queryParams1
buslocation_k = url_k + queryParams_k
#'http://openapi.gbis.go.kr/ws/rest/buslocationservice?serviceKey=M%2B4%2FqUiadT8X8PhgFjaQLDu%2BIOgPMURfGsOX%2FmVxwHQVJgnVR%2FMPjDYXkuQNwUFbZXlfnX5Lls3SUCiCLIFjgQ%3D%3D&routeId=233000031'
#url_k + queryParams_k

request_location = Request(buslocation)
request_location.get_method = lambda: 'GET'
response_location= urlopen(request_location).read().decode('utf8')

request_location_k = Request(buslocation_k)
request_location_k.get_method = lambda: 'GET'
response_location_k = urlopen(request_location_k).read().decode('utf8')

request_arrive = Request(busarrive)
request_arrive.get_method = lambda: 'GET'
response_arrive = urlopen(request_arrive).read().decode('utf8')

In [199]:
## bus 별로 정렬
s = response_location_k.split('<busLocationList>')
for x in range(len(s)) :
    bus_list = s[x]

## bus 수 count
sum = 0
for x in range(len(s)) :
    sum += s[x].count('<busType>')

In [195]:
## query time
a = s[0].split('>')
b = a[10]
query_time = b[:-11]

In [201]:
s.pop(0)
for line in s :
    endBus = []
    lowPlate = []
    plateNo = []
    plateType = []
    remainSeatCnt = []
    routeId = []
    stationId = []
    stationSeq = []
    
    line.split('>')
    print(line)

<endBus>0</endBus><lowPlate>0</lowPlate><plateNo>경기77바1097</plateNo><plateType>3</plateType><remainSeatCnt>28</remainSeatCnt><routeId>204000046</routeId><stationId>277103392</stationId><stationSeq>37</stationSeq></busLocationList>
<endBus>0</endBus><lowPlate>0</lowPlate><plateNo>경기70아8629</plateNo><plateType>3</plateType><remainSeatCnt>34</remainSeatCnt><routeId>204000046</routeId><stationId>206000017</stationId><stationSeq>45</stationSeq></busLocationList>
<endBus>0</endBus><lowPlate>0</lowPlate><plateNo>경기70아7474</plateNo><plateType>3</plateType><remainSeatCnt>39</remainSeatCnt><routeId>204000046</routeId><stationId>277103441</stationId><stationSeq>14</stationSeq></busLocationList>
<endBus>0</endBus><lowPlate>0</lowPlate><plateNo>경기70아7361</plateNo><plateType>3</plateType><remainSeatCnt>45</remainSeatCnt><routeId>204000046</routeId><stationId>100000001</stationId><stationSeq>24</stationSeq></busLocationList>
<endBus>0</endBus><lowPlate>0</lowPlate><plateNo>경기70아7482</plateNo><plateTy